___

<center><h1>Análisis Posicionamiento Youtube</h1></center>

___

<center><h2>Pragma</h2></center><br>
<center> Assessment Consultor Cientifico de Datos </center>

___
<p></p>
<center style="color: #AA6373; font-weight: 400;"><strong>Presentado por:</strong></center>
<center style="color: #AA6373; font-weight: 400;">Jorge Forero L.</center>

<center>Mayo 2025</center>
<p></p>

In [3]:

# --- Common Modules ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import joblib

In [4]:
df = pd.read_csv('../data/cleaned_youtube_data.csv')

## Feature Engineering

In [5]:
df = pd.read_csv('../data/cleaned_youtube_data.csv')

In [6]:
# 1. Time-based features
df['publish_hour'] = pd.to_datetime(df['publish_time']).dt.hour
df['publish_day'] = pd.to_datetime(df['publish_time']).dt.day_name()
df['days_to_trend'] = (pd.to_datetime(df['trending_date']) - pd.to_datetime(df['publish_time'])).dt.days

In [7]:
# 2. Engagement metrics
df['engagement_rate'] = (df['likes'] + df['dislikes'] + df['comment_count']) / df['views']
df['like_ratio'] = df['likes'] / (df['likes'] + df['dislikes'])

In [8]:
# --- Model Development ---
# 1. Define features and target
features = ['category_id', 'publish_hour', 'days_to_trend', 'comments_disabled', 
           'ratings_disabled', 'video_error_or_removed']
target = 'views'  # or 'engagement_rate' depending on the goal

In [9]:
# 2. Split the data
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# 3. Create preprocessing pipeline
numeric_features = ['publish_hour', 'days_to_trend']
categorical_features = ['category_id', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [12]:
# 4. Model Pipeline
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
}

In [13]:
# 5. Model Training and Evaluation
results = {}
for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = pipeline.predict(X_test)
    
    # Calculate metrics
    results[name] = {
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred)
    }

In [15]:
# 6. Feature Importance Analysis
# For Random Forest and XGBoost models
feature_importance = {}
for name, model in models.items():
    if name in ['Random Forest', 'XGBoost']:
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        pipeline.fit(X_train, y_train)
        
        # Get feature names after preprocessing
        numeric_feature_names = numeric_features
        categorical_feature_names = pipeline.named_steps['preprocessor']\
            .named_transformers_['cat']\
            .get_feature_names_out(categorical_features)
        
        # Combine feature names properly using numpy concatenate
        feature_names = np.concatenate([numeric_feature_names, categorical_feature_names])
        
        # Get feature importance
        if name == 'Random Forest':
            importance = pipeline.named_steps['model'].feature_importances_
        else:
            importance = pipeline.named_steps['model'].feature_importances_
        
        feature_importance[name] = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)

In [16]:
# 7. Model Persistence
# Save the best performing model
best_model_name = min(results, key=lambda x: results[x]['RMSE'])
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', models[best_model_name])
])
best_pipeline.fit(X_train, y_train)

# Save the model
joblib.dump(best_pipeline, '../models/best_model.joblib')

FileNotFoundError: [Errno 2] No such file or directory: '../models/best_model.joblib'